In [ ]:
!pip install python_speech_features
!pip install pydub

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5870 sha256=8c99a5107a5586d2199aff82a82fe4ab9a24c3592dcd9eed45f4356935cd6d44
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os

def set_seed(seed_value=42):
    """Set the random number generator seed for all modules."""
    random.seed(seed_value)  # Built-in Python random module
    np.random.seed(seed_value)  # NumPy
    torch.manual_seed(seed_value)  # PyTorch function for CPUs
    torch.cuda.manual_seed(seed_value)  # PyTorch function for GPUs
    torch.cuda.manual_seed_all(seed_value)  # PyTorch function for multi-GPUs
    torch.backends.cudnn.deterministic = True  # Makes cudnn algorithm deterministic
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_seed(42)  # Call with the desired seed value

In [ ]:
import numpy as np
#import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from IPython.lib.display import Audio
from scipy.fftpack import fft, ifft
from scipy.io import loadmat
import scipy.signal as sgnl
import scipy.io.wavfile as wav
import sys
import wave
import operator
import scipy
from python_speech_features import mfcc

In [ ]:
import torch
import torch.nn as nn
# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Set the current directory
%cd /content/drive/My Drive/birdcall

Mounted at /content/drive
/content/drive/My Drive/birdcall


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.io import wavfile
from python_speech_features import mfcc
import os
from scipy.io import wavfile, loadmat
from scipy.fft import fft, fftfreq, ifftshift
from scipy.fftpack import fftshift
from python_speech_features import mfcc
import torch
from torch.utils.data import TensorDataset, DataLoader

# Label to index mapping dictionary as provided
labels_index = {
    'Velvet_Scoter_Melanitta_fusca': 0,
    'Long_tailed_Duck_Clangula_hyemalis': 1,
    'Leach\'s_Storm_Petrel_Hydrobates_leucorhous': 2,
    'Horned_Grebe_Podiceps_auritus': 3,
    'Great_Bustard_Otis_tarda': 4,
    'European_Turtle_Dove_Streptopelia_turtur': 5,
    'Black_legged_Kittiwake_Rissa_tridactyla': 6,
    'Balearic_Shearwater_Puffinus_mauretanicus': 7,
    'Atlantic_Puffin_Fratercula_arctica': 8,
    'Aquatic_Warbler_Acrocephalus_paludicola': 9
}

# Invert the dictionary to map indices to labels
index_to_labels = {v: k for k, v in labels_index.items()}

# Define the model class
class LSTMSoundClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=3):
        super(LSTMSoundClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size * 2)
        self.fc2 = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

def apply_fourier_transform(input_audio, Fs):
    N = len(input_audio)
    f_transform = fftshift(fft(input_audio, N))
    frequencies = np.linspace(-Fs/2, Fs/2, N)
    return f_transform, frequencies

import scipy.signal as signal

def apply_lowpass_filter(input_audio, b, a):
    filtered_audio = signal.lfilter(b, a, input_audio)
    return filtered_audio

def extract_mfcc_features(filtered_audio, Fs):
    mfcc_features = mfcc(filtered_audio, Fs)
    return mfcc_features

# Define preprocessing function
def preprocess_audio(file_path, b, a):
    # Load WAV file
    Fs, input_audio = wavfile.read(file_path)

    # Apply low-pass filtering
    filtered_audio = apply_lowpass_filter(input_audio, b, a)

    # Extract MFCC
    mfcc_features = mfcc(filtered_audio, samplerate=Fs, numcep=13)

    return mfcc_features


# Define the hyperparameters for the model to use
num_classes = 10  # Assuming there are 10 classes. Please change it to the actual number of classes.
input_size = 13  # Size of MFCC features (must match numcep value)
hidden_size = 512
num_layers = 3

# Create model instance and load trained weights
model = LSTMSoundClassifier(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes, num_layers=num_layers)
model.load_state_dict(torch.load('./3.23.pth'))  # Modify 'model_path.pth' to your model's path.
model.eval()

# Load filter coefficients - Set the path to 'highpass(500).mat' file accordingly
filter_data = loadmat('./low_filter/highpass(500).mat')  # Modify 'filter_path.mat' to your filter's path.
Coeffs = filter_data['ba'].astype(np.float64)
b = Coeffs[0, :]
a = 1

# Set the path for the audio file to infer
test_file_path = './test/Velvet_Scoter_TEST.wav'  # Set the test audio file path.

# Preprocess the audio file
test_feature = preprocess_audio(test_file_path, b, a)
test_feature_tensor = torch.tensor([test_feature], dtype=torch.float)


# Execute inference
with torch.no_grad():
    output = model(test_feature_tensor)
    probabilities = torch.softmax(output, dim=1)
    predicted_index = probabilities.argmax(dim=1).item()
    predicted_prob = probabilities.max(dim=1).values.item()

    # Use the index to get the associated label from our dictionary
    predicted_label = index_to_labels[predicted_index]

    print(f"Predicted label: {predicted_label}, Probability: {predicted_prob}")

Predicted label: Long_tailed_Duck_Clangula_hyemalis, Probability: 0.8901532888412476
